In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


In [2]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [3]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [4]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    print(p)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [5]:
#Get repond according to questions

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [6]:
#chatbot res

def chatbot_response(msg):
    respout=["See you!", "Have a nice day", "Bye! Come back again soon."]
    ints = predict_class(msg, model)
    print(ints)
    res = getResponse(ints, intents)
    print(res)
    ans=res in respout
    ans
    return res

In [7]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    print("msg::",msg)
    EntryBox.delete("0.0",END)
    

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "ANA: " + res + '\n\n')
        if res=='exit':
            base.destroy()
        else:
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
            
        

In [8]:
#Creating Bot GUI with tkinter
import tkinter
from tkinter import *
base = Tk()
base.title("ANA")
base.iconbitmap('bot.ico')
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
base.configure(bg='#10C8F6')

#Create Chat window
ChatLog = Text(base, bd=0, bg="#DAECEE", height="8", width="50", font="Times")
ChatLog.insert(END, "ANA: " + "Hi i'm your assistant here to help you" + '\n\n')
ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

ChatLog.config(state=DISABLED)

#Add scrollbar to Chat window/ChatLog
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="arrow")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#0F10F4", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the Entry box
EntryBox = Text(base, bd=0, bg="#DAECEE",width="29", height="5", font="Arial")




#Place all widget on the screen y start from up 
scrollbar.place(x=376,y=6,height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=6, y=400, height=90, width=270)
SendButton.place(x=285,y=400,height=90,width=106)



base.mainloop()

msg:: Your name ?
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[{'intent': 'Name', 'probability': '1.0'}]
My name is Ana
